## Cafe sales analysis
By Alan Tolubayev

[Cafe Sales](https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training) dataset was used for analysis.

#### Business Context

As a data science consultant specializing in retail analytics for small to medium-sized businesses, I was approached by a Thailand-based cafe to analyze their sales for the past year and help them to boost revenue.

#### Objective
The cafe has collected transactional data, however, they lack the expertise to extract meaningful insights from it. As a data scientist, my role is to help with insights.
1. What are the limitations of the data they provided?
2. What are the top selling items?
3. How do sales behave month over month?
4. What strategies can maximize profitability?

## Start Here
The code below will check for missing libraries and install them. Make sure the requirements file is located in the same folder as the analysis file.

In [ ]:
import importlib
import os

def install_required_packages():
    required_packages = [
        "pandas",
        "numpy",
        "matplotlib",
        "seaborn",
    ]
    
    missing_packages = []
    for package in required_packages:
        pkg_name = package.split('==')[0]  # in case you add versions later
        if importlib.util.find_spec(pkg_name) is None:
            missing_packages.append(package)

    if missing_packages:
        print(f"🔍 Installing missing packages: {missing_packages}")
        for pkg in missing_packages:
            if os.system(f"pip install {pkg}") != 0:
                print(f"❌ Failed to install {pkg}")
        print("✅ Installation complete.")
    else:
        print("✅ All required packages are already installed.\nYou're good to go!")

# Run the installer
install_required_packages()

## Data Import and Cleaning

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

def load_data(file, delimiters=[',', ';', '\t', '|']):
    for sep in delimiters:
        try:
            df = pd.read_csv(file, sep=sep, encoding='utf-8')
            if not df.empty:
                print(f"✅ Loaded data using '{sep}' delimiter.\nPreview:")
                print(df.head())
                return df
        except Exception:
            continue
    print("❌ Failed to load data. Check the file and delimiters.")
    return None
raw_df = load_data('cafe_sales.csv')

cleaning_df = raw_df.copy()

print(f"Duplicate rows: {cleaning_df.duplicated().sum()}")
print(f'\nMissing values count:\n{cleaning_df.isnull().sum()}')  # Count missing values per column
print(f'\nMissing values sample:\n{cleaning_df[cleaning_df.isnull().any(axis=1)]}')  # Show rows with missing values

**Observations:**
1. **Transaction ID** is the primary key in our exercise. We have 10,000 unique transactions. We will use this number as the baseline for other columns.
2. Item represents sold items in the order. We have 9,667 values. That means we are missing 333 values.
3. Quantity column is missing 138 values.
4. **Price per unit** is misssing 179 values. This column represents a monetary value of an item sold. I will review the Items column to find the prices, create a dictionary with "Item" : "Price per unit" values and replace the missing values.
5. **Total Spent** is a calculated column. It supposed to be a multiplication of quantity and price per unit. I will use this logic to recalculate and review the column.
5. Payment methods also missing 2,579 values (~26%). This column is one of the highest missing value column. We will investigate it futher.
6. Location. 3,265 missing values (~33%). We will also investigate this column futher.
7. Transaction date. Missing 159 values.

**Next steps:**
1. Changing data types to make sure the items are strigns and prices represent float values.
1. Replacing "UNKNOWN" values with in transaction dates and checking why it happens with a client.
1. Impude missing values to avoid creating a biased dataset.

In [ ]:
cleaning_df = cleaning_df.replace(['ERROR', 'UNKNOWN', np.nan], '') # Replace ERROR values with empty strings to clean later

columns_to_int = ['Quantity']
columns_to_float = ['Price Per Unit', 'Total Spent']

for col in columns_to_int:
    cleaning_df[col] = pd.to_numeric(cleaning_df[col], errors='coerce').fillna(0).astype(int)

for col in columns_to_float:
    cleaning_df[col] = pd.to_numeric(cleaning_df[col], errors='coerce').fillna(0).astype(float)

Now we need to check and impude missing data in "Item", "Payment Method", "Location", and "Transaction Date" columns. In the original dataset we saw some Errors and missing values. Since we converted all Errors and Unknown values to empty strings, after conversion they became 'NaT' datatypes which means 'Not a Time'. To find them we will use [isna](https://pandas.pydata.org/docs/reference/api/pandas.isna.html) function.

Dates are important indicators for pattern analysis. In order to analyze the data futher we will use Python's mode function to find the most frequent date and replace the NaT values with it. We will use Panda's [pd.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) and [dt.normalize](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.normalize.html) function to fix the missing transaction dates.

In [ ]:
columns_to_check = {
    'Item': '',
    'Payment Method': '',
    'Location': '',
    'Transaction Date': pd.NaT  # Use NaT for dates
}

total_values = cleaning_df['Transaction ID'].count()
print(f"Total number of values: {total_values}")

for col, missing_value in columns_to_check.items():
    if missing_value == '':
        missing_count = cleaning_df[cleaning_df[col] == '']['Transaction ID'].count()
    else:
        missing_count = cleaning_df[cleaning_df[col].isna()]['Transaction ID'].count()

    missing_ratio = missing_count / total_values * 100
    print(f"Number of missing {col.lower()}: {missing_count}")
    print(f"Missing {col.lower()} ratio: {missing_ratio:.2f}%")

def clean_column(df, column_name):
    df[column_name] = df[column_name].replace('', 'Other')
    replaced_count = df[df[column_name] == 'Other']['Transaction ID'].count()
    print(f'{column_name} replaced with "Other": {replaced_count}')
    return df

import pandas as pd
import numpy as np

def clean_transaction_date(df):
    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce').dt.normalize()

    df = df[df['Transaction Date'].notna()]

    print(f"Number of missing dates after cleaning: {df['Transaction Date'].isna().sum()}")

    return df

columns_to_clean = ['Item', 'Payment Method', 'Location']
for column in columns_to_clean:
    cleaning_df = clean_column(cleaning_df, column)

cleaning_df = clean_transaction_date(cleaning_df)

Checking for consistency in numerical data:
- There are items where Price Per Unit is zero
- In some cases Quantity is zero
- Total spent is zero

To fix this issue we will create a dictionary of iteams and their prices and make sure all the itesm have a price.

In [ ]:
ppu = cleaning_df[cleaning_df['Price Per Unit'] == 0]['Transaction ID'].count()
qty = cleaning_df[cleaning_df['Quantity'] == 0]['Transaction ID'].count()
total = cleaning_df[cleaning_df['Total Spent'] == 0]['Transaction ID'].count()
print(f'Missing prices per unit: {ppu}')
print(f'Missing quantity: {qty}')
print(f'Missing totals spent: {total}')

To fix the inconsistency in the transactional data we will create a dictionary with prices for items.
And implement a function that will check for Total Spent is above zero.
If the total spent doesn't equal to Price Per Item multiplied by Quantity:
1. We look for columns where Price per Unit or Quantity are zero and replace the Price Per Unit from the dictionary
2. We look for Total Spent and Price per Item to recalculate Quantity column by dividing Total Spent by Price Per Item

In [ ]:
item_price_dict = (
    cleaning_df[cleaning_df['Price Per Unit'] > 0]  # Keep only prices > 0
    [['Item', 'Price Per Unit']]
    .drop_duplicates()
    .set_index('Item')['Price Per Unit']
    .to_dict()
)

print("Item-Price Dictionary:")
print(item_price_dict)

import pandas as pd

def fix_inconsistent_transactions(df, price_dict):
    df = df.copy()

    def fix_row(row):
        item = row['Item']
        price = row['Price Per Unit'] or price_dict.get(item, 0)
        qty = row['Quantity']
        total = row['Total Spent']

        if price == 0:
            price = price_dict.get(item, price)

        if qty == 0:
            qty = total / price if total != 0 and price != 0 else 1

        if total == 0 or total != qty * price:
            total = qty * price

        row['Price Per Unit'], row['Quantity'], row['Total Spent'] = price, qty, total
        return row

    return df.apply(fix_row, axis=1)

cleaning_df = fix_inconsistent_transactions(cleaning_df, item_price_dict)

inconsistent_rows = cleaning_df[ 
    (cleaning_df['Total Spent'] != cleaning_df['Quantity'] * cleaning_df['Price Per Unit'])
]
print(f"Remaining inconsistent transactions: {inconsistent_rows.shape[0]}")

In [ ]:
pay_methods = cleaning_df['Payment Method'].unique()
locations = cleaning_df['Location'].unique()
duplicates = cleaning_df.duplicated().sum()

pay_methods_str = ", ".join(pay_methods)
locations_str = ", ".join(locations)

print(f"Unique payment methods: {pay_methods_str}")
print(f"Unique locations: {locations_str}")
print(f"Number of duplicate rows: {duplicates}")

inconsistent_rows = cleaning_df[ 
    cleaning_df['Total Spent'] != cleaning_df['Quantity'] * cleaning_df['Price Per Unit']
]

if inconsistent_rows.empty:
    print("Inconsistent data: None found")
else:
    print("Inconsistent data:")
    print(inconsistent_rows)

## Exploratory Data Analysis

### Question 1: What are the limitations of the data?

Despite my best effort, there are still limitations to the dataset that we should be aware of:
1. Missing and Imputed data. 30% of payment data and 37.8% of location data were imputed during cleaning. Unfortunately we won't be able to use it for analysis.
2. Missing client data. Without client data I won't be able to track loyalty, repeat purchases, or provide a segmentation analysis.
3. Low depth of data. Without margins and expenses I won't be able to make a Profit and Loss analysis.

In [ ]:
clean_df = cleaning_df.copy()
print(f"Replaced payment method data: {clean_df[clean_df['Payment Method'] == 'Other']['Transaction ID'].count()}")
print(f"Replaced location data: {clean_df[clean_df['Location'] == 'Other']['Transaction ID'].count()}")
print(f"Replaced Item data: {clean_df[clean_df['Item'] == 'Other']['Transaction ID'].count()}")

### Question 2: What are the top selling items?

Identifiying top selling items would help to prioritize best performers and create bundles or combos to increace Average Order Value.
Calculating top selling items is done by grouping items by quantity and order value. To prevent bias towards the quantity of items, I'm adding a sum of total spent.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

top_selling_items = clean_df.groupby('Item')[['Quantity', 'Total Spent']].sum().sort_values(by='Quantity', ascending=False)

# Plotting top 10 items by Quantity
top_5_items = top_selling_items.head(5)

plt.figure(figsize=(8,4))
bar_plot = sns.barplot(data=top_5_items, x='Item', y='Quantity', hue='Total Spent', legend=False, palette='viridis')

# Annotate each bar with Total Spent values
for index, row in top_5_items.iterrows():
    bar_plot.text(index, row['Quantity'], f"{row['Total Spent']}", color='white', ha="center", va="bottom", fontsize=8, fontweight='bold')

plt.title("Top 5 selling items by quantity", fontsize=12)
plt.xlabel("", fontsize=10)
plt.ylabel("Total quantity sold", fontsize=10)
plt.show()

Coffee and juice are the best performing items on the menu, but we can also see food items being right after. I'd recommend creating a coffee + sandwich combo or salad + juice combo to increase the Average Order Value and sales performance during the day. Unfortunately we don't have time data during the day to check when people are more likely to buy these items.

### Question 3: How do sales behave month over month?

#### Sales by month
Helps to identify seasonal trends and busy periods to optimize stocks and plan promotions around peak months.

In [ ]:
clean_df['Month'] = clean_df['Transaction Date'].dt.month_name()

monthly_sales = clean_df.groupby('Month')['Total Spent'].sum()

month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

monthly_sales = monthly_sales.reindex(month_order).dropna().reset_index()

plt.figure(figsize=(12,4))
bar_plot = sns.barplot(data=monthly_sales, x='Month', y='Total Spent', palette='crest', hue='Total Spent', legend=False)

for idx, row in monthly_sales.iterrows():
    bar_plot.text(
        idx, 
        row['Total Spent'], 
        f"${row['Total Spent']:.2f}", 
        color='white', 
        ha="center", 
        va="bottom", 
        fontsize=8, 
        fontweight='bold'
    )

plt.title("Sales by Month", fontsize=12)
plt.xlabel("")
plt.ylabel("Revenue", fontsize=10)
plt.show()

Analysis identified February, July, September, and November as months with notably lower sales performance compared to other months. To stabilize revenue, we recommend launching targeted promotional campaigns during these months.

**Potential promotional strategies include:**	
- Limited offera to encourage more frequent visits or increase spending per visit.
- Product combos with attractive prices to boost sales.
- Introduce rewards that customers can earn in these lower-performing periods.
- Utilize social media to highlight promotions specifically for these months.

Implementing these promotional tactics should drive increased traffic and sales during traditionally slow periods, improving cafe’s overall revenue consistency.

#### Average Order Value over time
Heps to identify months will lower AOV and plan promotions around them.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

clean_df['Month'] = clean_df['Transaction Date'].dt.month_name()

monthly_aov = clean_df.groupby('Month')['Total Spent'].mean()

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']
monthly_aov = monthly_aov.reindex(month_order).dropna()

# Plot Monthly AOV
plt.figure(figsize=(12,5))
sns.barplot(x=monthly_aov.index, y=monthly_aov.values)

# Annotate bars clearly with values
for idx, value in enumerate(monthly_aov):
    plt.text(idx, value, f"{value:.2f}", ha='center', va='bottom', fontsize=10, fontweight='bold')

# Titles and labels
plt.title('Average Order Value month-over-month', fontsize=16)
plt.xlabel('')
plt.ylabel('\nAverage Order Value\n', fontsize=12)
plt.xticks()
plt.tight_layout()
plt.show()

### Question 4: What strategies can maximize profitability?

To maximize profitability I suggest the cafe to: 
1. Optimize offerings, like coffee and juice.
2. Create combos (coffee + sandwich) to increase the Average Order Value.
3. Start promo-campaigns around the low performing months like February, July, and September
4. Consider investing into a loyalty program that will increase the Average Order Value and attract new customers.